# Practical Guide to PCA
#### Gabriel Hassler


## PCA

In [ ]:
import pandas as pd

data = pd.read_csv('data/processed/pca_input.csv', dtype={'GEOID': str})

print(data.head())



## Attempt 1

In [ ]:
from sklearn.decomposition import PCA

X = data.drop(columns=['GEOID'])
X.index = data['GEOID']
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X)

## Attempt 2: Dealing with missing data

In [ ]:
mis_percs = X.isnull().mean()
print(mis_percs)


# plot missing data matrix
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,6))
sns.heatmap(X.isnull(), cbar=True, cbar_kws={'label': 'Missing Value Intensity'})
plt.title('Missing Data Matrix')
plt.xlabel('Features')
plt.ylabel('Samples')
plt.show()

In [ ]:
# drop missing values
X_dropped = X.dropna()
pca_dropped = PCA(n_components=10)
X_pca_dropped = pca_dropped.fit_transform(X_dropped)

# Eigenvectors as matrix
print("PCA Components after dropping missing values:")
print(pca_dropped.components_)

This is a little hard to interpret.
Let's make a function to visualize the PCA components.

In [ ]:
def visualize_pca(pca, feature_names):
    components = pca.components_
    num_components = components.shape[0]

    plt.figure(figsize=(12, num_components * 5))  # Adjust height based on number of components
    for i in range(num_components):
        plt.subplot(num_components, 1, i + 1)  # Change subplot configuration
        plt.bar(range(len(feature_names)), components[i])
        plt.xticks(range(len(feature_names)), feature_names, rotation=90)
        plt.ylim(-1, 1)
        plt.axhline(0, color='gray', linestyle='dotted')  # Add a dotted line at y=0
        plt.title(f'PCA Component {i+1}')
    plt.tight_layout()
    plt.show()

def visualize_eigvals(pca):
    eigvals = pca.explained_variance_
    plt.figure(figsize=(8, 5))
    plt.plot(range(1, len(eigvals) + 1), eigvals, marker='o')
    plt.title('Eigenvalues of PCA Components')
    plt.xlabel('Component Number')
    plt.ylabel('Eigenvalue')
    plt.xticks(range(1, len(eigvals) + 1))
    plt.grid()
    plt.show()

visualize_pca(pca_dropped, X.columns)
visualize_eigvals(pca_dropped)

PCA tries to find the directions of maximum variance in the data.
Let's look at the variance of each variable in our dataset.

In [ ]:
var_X = X_dropped.var()
print("Variance of each variable:")
print(var_X)

## Attempt 3: Dealing with scaling



In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_dropped)


var_X_scaled = pd.DataFrame(X_scaled, columns=X_dropped.columns).var()
print("Variance of each variable after scaling:")
print(var_X_scaled)


In [ ]:
pca_scaled = PCA(n_components=10)
X_pca_scaled = pca_scaled.fit_transform(X_scaled)

print("PCA Components after scaling:")
visualize_pca(pca_scaled, X_dropped.columns)
print("Eigenvalues after scaling:")
visualize_eigvals(pca_scaled)

## How to choose the number of components to include in downstream analyses?
1. Scree plot
2. Cumulative explained variance plot
3. Kaiser criterion (keep components with eigenvalues > 1)
4. Cross validation
5. Domain knowledge

### Scree Plot

In [ ]:
visualize_eigvals(pca_scaled)


### Cumulative Explained Variance Plot



In [ ]:
var_explained = pca_scaled.explained_variance_ratio_
cumulative_var_explained = var_explained.cumsum()

df = pd.DataFrame({
    'Component': range(1, len(var_explained) + 1),
    'Variance Explained': var_explained,
    'Cumulative Variance Explained': cumulative_var_explained
})
print(df)

### Kaiser Criterion
Keep components with eigenvalues > 1.

In [ ]:
eigvals = pca_scaled.explained_variance_
print("Eigenvalues of PCA Components after scaling:")
print(eigvals)

### Cross Validation
This is complicated in PCA but doable.

### Domain Knowledge
Use your understanding of the data to inform your choice.

## Advanced PCA Topics: Rotation and Sparse PCA

In [ ]:
# sparse PCA
from sklearn.decomposition import SparsePCA
sparse_pca = SparsePCA(n_components=3, alpha=10)
sparse_X_pca = sparse_pca.fit_transform(X_scaled)

visualize_pca(sparse_pca, X_dropped.columns)

In [ ]:
from sklearn.decomposition import FactorAnalysis

fa = FactorAnalysis(n_components=3, rotation='quartimax')
X_fa = fa.fit_transform(X_scaled)

visualize_pca(fa, X_dropped.columns)

# Interpretation

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely import affinity

# Load county shapefile
counties = gpd.read_file("data/raw/cb_2022_us_county_5m.zip")

# Project to an equal-area projection in meters
counties = counties.to_crs("EPSG:5070")

# Build PCA dataframe
X_pca_df = pd.DataFrame(X_pca_scaled, columns=[f'PC{i+1}' for i in range(X_pca_scaled.shape[1])])
X_pca_df['GEOID'] = X_dropped.index

# Separate Alaska, Hawaii, and continental US
alaska = counties[counties['STATEFP'] == '02'].copy()
hawaii = counties[counties['STATEFP'] == '15'].copy()
continental = counties[~counties['STATEFP'].isin(['02', '15', '72'])].copy()

# --- Move Alaska as a block ---
# Compute Alaska centroid to use as scaling origin
alaska_centroid = alaska.unary_union.centroid

# Scale and shift each geometry with the same parameters
alaska['geometry'] = alaska['geometry'].apply(
    lambda geom: affinity.scale(geom, xfact=0.35, yfact=0.35, origin=alaska_centroid)
)
alaska['geometry'] = alaska['geometry'].apply(
    lambda geom: affinity.translate(geom, xoff=3e6, yoff=-1.5e6)
)

# --- Move Hawaii as a block ---
hawaii_centroid = hawaii.unary_union.centroid
hawaii['geometry'] = hawaii['geometry'].apply(
    lambda geom: affinity.translate(geom, xoff=5.4e6, yoff=-1.8e6)
)

# Combine all back together
counties_shifted = pd.concat([continental, alaska, hawaii], ignore_index=True)

# Merge with PCA data
merged = counties_shifted.merge(X_pca_df, on='GEOID', how='left')

# --- Plot each PCA component ---
for i in range(4):
    pc = f'PC{i+1}'
    fig, ax = plt.subplots(figsize=(12, 9))
    merged.plot(column=pc, ax=ax, legend=True,
                cmap='viridis', edgecolor='black', linewidth=0.1)
    ax.set_title(f"{pc} by County (shifted AK/HI)", fontsize=14)
    ax.axis('off')
    plt.show()


In [ ]:
merged

